# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [21]:
import openai
import pandas as pd

#OpenAI API key
api_key = 'sk-KuqOZZy_EbVm4qVg-Eh1ij4rTpGgR-DXMw1vT8N04rT3BlbkFJrsEyLcAkrPxORfSxyNwPz7LzK01eHka_a7TvW0SbQA'






In [22]:
## Create the assistant
from openai import OpenAI
 
client = OpenAI(api_key=api_key)
 
assistant = client.beta.assistants.create(
  name="ChatGPT prompt expert",
  instructions="You are an expert in ChatGPT prompts for different sectors",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
)

In [23]:
# Create a vector store caled "Datavisualization Documents"
vector_store = client.beta.vector_stores.create(name="Prompt Documents")

In [24]:
 # Ready the files for upload to OpenAI
file_paths = ["chatgpt_prompts.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

In [25]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [26]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [27]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "give me prompts for Aircraft Manufacturing",
        }
    ]
)

# Run the thread and poll for the response
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# Retrieve messages from the thread
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

# Check if there are any messages in the response
if messages and messages[0].content:
    # Extract the text content of the first message
    message_content = messages[0].content[0].text
    print(message_content)
else:
    print("No content found in the messages error message.")


Text(annotations=[], value='Here are some prompts tailored for the Aircraft Manufacturing sector:\n\n1. **Design and Development**:\n   - "Describe the steps involved in the aircraft design process, from conceptual design to final production. What are the key challenges faced at each stage?"\n   - "How do innovations in materials science impact aircraft manufacturing, particularly in terms of weight reduction and fuel efficiency?"\n\n2. **Regulatory and Compliance**:\n   - "What are the major regulatory challenges aircraft manufacturers face in different regions? How can companies ensure compliance while maintaining efficiency?"\n   - "Explain the importance of safety certifications in aircraft manufacturing and the processes involved in obtaining them."\n\n3. **Production and Operations**:\n   - "Discuss the role of lean manufacturing principles in aircraft production. How can manufacturers minimize waste while ensuring high-quality output?"\n   - "What are the latest trends in automa

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [ ]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality